In [1]:
!pip install numpy pandas scikit-learn
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [1]:
# Data Loading and Preprocessing
import pandas as pd
import glob

# Path to your folder
path = '../../data/training/labled_data'

# Find all CSV files in the folder
all_files = glob.glob(path + "/Tracking_video*_labled.csv")

# List to store the dataframes
li = []

# Read each CSV file and add it to the list
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Combine all dataframes in the list into a single dataframe
data = pd.concat(li, axis=0, ignore_index=True)

In [3]:
# Model Training
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer


data = data[['Left hip_y', 'Right hip_y', 'Label']]

# Features
X = data.drop('Label', axis=1)
# Labels
y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use mean imputation for missing values
imputer = SimpleImputer(strategy='mean')

# Fit on the training data and transform both training and test data
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)
# Initialize the model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

RandomForestClassifier()

In [4]:
# Train multiple models and compare the accuracies

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

data = data[['Left hip_y', 'Right hip_y', 'Label']]

# Features
X = data.drop('Label', axis=1)
# Labels
y = data['Label']

# Calculate the difference between the current and previous row for 'Left hip_y' and 'Right hip_y'
X['Left hip_y_diff'] = X['Left hip_y'].diff()
X['Right hip_y_diff'] = X['Right hip_y'].diff()

# Drop rows with missing values in X and the corresponding rows in y
na_indices = X.dropna().index
X = X.loc[na_indices]
y = y.loc[na_indices]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize the scaler
scaler = MinMaxScaler()

# Fit on the training data and transform both training and test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Use mean imputation for missing values
imputer = SimpleImputer(strategy='mean')

# Fit on the training data and transform both training and test data
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Initialize the models
models = [
    RandomForestClassifier(),
    LogisticRegression(),
    SVC(),
    DecisionTreeClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    MLPClassifier(max_iter=1000)
]

# Train and evaluate each model
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")

RandomForestClassifier Accuracy: 0.863349131121643
LogisticRegression Accuracy: 0.7466711803204694
SVC Accuracy: 0.8099751749040849
DecisionTreeClassifier Accuracy: 0.8108779056646355
GradientBoostingClassifier Accuracy: 0.8469871360866622
KNeighborsClassifier Accuracy: 0.8737305348679756
MLPClassifier Accuracy: 0.8253215978334462


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

# Assuming 'data' has been preprocessed to include only ['Left hip_y', 'Right hip_y', 'Label']

# Function to create sequences from data
def create_sequences(features, labels, window_size):
    feature_sequences, label_sequences = [], []
    for i in range(len(features) - window_size + 1):
        feature_sequences.append(features.iloc[i:(i + window_size)].values)
        label_sequences.append(labels.iloc[i + window_size - 1])
    return np.array(feature_sequences), np.array(label_sequences)

# Preparing the data
window_size = 4  # This is the customizable window size

# Calculate the difference between the current and previous row for 'Left hip_y' and 'Right hip_y'
data['Left hip_y_diff'] = data['Left hip_y'].diff().fillna(0)
data['Right hip_y_diff'] = data['Right hip_y'].diff().fillna(0)

# Features and Labels
# X = data[['Left hip_y', 'Right hip_y', 'Left hip_y_diff', 'Right hip_y_diff']]
X = data[['Left hip_y', 'Right hip_y']]
y = data['Label']

# Create sequences
X_seq, y_seq = create_sequences(X, y, window_size)

# Split the data into training and testing sets
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Flatten the sequences for compatibility with traditional models
X_train_flat = X_train_seq.reshape(X_train_seq.shape[0], -1)
X_test_flat = X_test_seq.reshape(X_test_seq.shape[0], -1)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit on the training data and transform both training and test data
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

# Use mean imputation for missing values (if necessary)
imputer = SimpleImputer(strategy='mean')

# Fit on the training data and transform both training and test data
X_train_imputed = imputer.fit_transform(X_train_scaled)
X_test_imputed = imputer.transform(X_test_scaled)

# Initialize the models
models = [
    RandomForestClassifier(),
    LogisticRegression(),
    SVC(),
    DecisionTreeClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    MLPClassifier(max_iter=1000)
]

# Train and evaluate each model
for model in models:
    model.fit(X_train_imputed, y_train_seq)
    y_pred = model.predict(X_test_imputed)
    accuracy = accuracy_score(y_test_seq, y_pred)
    print(f"{model.__class__.__name__} Accuracy: {accuracy}")

RandomForestClassifier Accuracy: 0.913806683920333
LogisticRegression Accuracy: 0.7752897490716777
SVC Accuracy: 0.8119725441656352
DecisionTreeClassifier Accuracy: 0.8694722628558569
GradientBoostingClassifier Accuracy: 0.799707437830539
KNeighborsClassifier Accuracy: 0.9005288623832565
MLPClassifier Accuracy: 0.8608079216833577


In [14]:
import joblib
import os

# Directory where you want to save the models
save_dir = "../../data/models/sequence"

# Create the directory if it does not exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Train and evaluate each model
for model in models:
    model_name = model.__class__.__name__

    # Save the model
    joblib.dump(model, os.path.join(save_dir, f"{model_name}.pkl"))

In [ ]:
# Model Evaluation
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

# Directory where the models are saved
save_dir = "../../data/models/bundled"

# Load the RandomForestClassifier model from the save_dir
model_path = os.path.join(save_dir, "RandomForestClassifier.pkl")
model = joblib.load(model_path)

# Predict the labels for the test set
y_pred = model.predict(X_test)

# Print the accuracy and the classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8649289099526066
              precision    recall  f1-score   support

   Ascending       0.87      0.83      0.85      2382
  Descending       0.87      0.88      0.87      2999
       Pause       0.86      0.88      0.87      3481

    accuracy                           0.86      8862
   macro avg       0.87      0.86      0.86      8862
weighted avg       0.87      0.86      0.86      8862



In [ ]:
import joblib
import os

# Directory where you want to save the model
save_dir = "../../data/models"

# Create the directory if it does not exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save the model to the specified directory
joblib.dump(model, os.path.join(save_dir, "RandomForest3onlyhips.pkl"))

['../../data/models/RandomForest3onlyhips.pkl']

In [ ]:
# Model Loading and Usage
import joblib
import os

# Directory where the models are saved
save_dir = "../../data/models"

# Load the model
model_path = os.path.join(save_dir, "DecisionTreeClassifier.pkl")
loaded_model = joblib.load(model_path)

# Use the loaded model to make predictions
# For example, predict the label for the first instance in the test set
single_prediction = loaded_model.predict([X_test.iloc[0]])
print("Prediction for the first instance in the test set:", single_prediction)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
import pandas as pd
import os
import joblib
from sklearn.impute import SimpleImputer

# Load the model
loaded_model = joblib.load('../../data/models/RandomForestClassifier.pkl')

# Load the fitted imputer
imputer = joblib.load('../../data/models/imputer.pkl')

# Path to the tracking data
tracking_data_path = '../../data/test/tracking_data'

# Path to save the new data with predictions
new_data_path = '../../data/test/predictions'

# Create the new data directory if it doesn't exist
if not os.path.exists(new_data_path):
    os.makedirs(new_data_path)

# Iterate over all the tracking data files in the path
for filename in os.listdir(tracking_data_path):
    if filename.endswith('.csv'):
        # Load new data
        new_data = pd.read_csv(os.path.join(tracking_data_path, filename))

        # If the new data includes the 'Label' column, drop it
        if 'Label' in new_data.columns:
            new_data = new_data.drop('Label', axis=1)

        # Select only the 'Left hip_y' and 'Right hip_y' columns
        new_data = new_data[['Left hip_y', 'Right hip_y']]

        # Transform the new data using the fitted imputer
        new_data = imputer.transform(new_data)

        # Use the loaded model to make predictions
        predictions = loaded_model.predict(new_data)

        # Add the predictions back into the 'Label' column
        new_data['Label'] = predictions

        # Save the DataFrame with the new labels back to a CSV file in the new data path
        new_data.to_csv(os.path.join(new_data_path, filename), index=False)

/Users/jonasgwozdz/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/jonasgwozdz/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/jonasgwozdz/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/jonasgwozdz/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/jonasgwozdz/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/jonasgwozdz/Librar

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Assuming 'data' is already loaded as in your example

# Encoding categorical labels to integers
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

# Features and labels
X = data.drop('Label', axis=1).values
y = data['Label'].values

# Imputation for missing values
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Reshape X to be 3D [samples, time steps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# One-hot encode the labels
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=1000, batch_size=1, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


Epoch 1/1000
35549/35549 [==============================] - 21s 570us/step - loss: 1.0145 - accuracy: 0.5162 - val_loss: 1.0202 - val_accuracy: 0.5149
Epoch 2/1000
35549/35549 [==============================] - 20s 551us/step - loss: 0.9963 - accuracy: 0.5285 - val_loss: 0.9974 - val_accuracy: 0.5204
Epoch 3/1000
35549/35549 [==============================] - 20s 558us/step - loss: 0.9922 - accuracy: 0.5304 - val_loss: 1.0004 - val_accuracy: 0.5169
Epoch 4/1000
35549/35549 [==============================] - 20s 555us/step - loss: 0.9894 - accuracy: 0.5302 - val_loss: 0.9906 - val_accuracy: 0.5249
Epoch 5/1000
35549/35549 [==============================] - 20s 556us/step - loss: 0.9855 - accuracy: 0.5305 - val_loss: 0.9864 - val_accuracy: 0.5273
Epoch 6/1000
35549/35549 [==============================] - 24s 679us/step - loss: 0.9781 - accuracy: 0.5323 - val_loss: 0.9770 - val_accuracy: 0.5262
Epoch 7/1000
35549/35549 [==============================] - 20s 569us/step - loss: 0.9707 - ac

KeyboardInterrupt: 